In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 

import tensorflow as tf
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
    
#unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-16 18:32:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-16 18:32:42 (77.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]

--2022-06-16 18:32:44--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 173.194.202.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [appl

In [2]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
df_train_shuffled = df_train.sample(frac=1, random_state=123)
df_train_shuffled.head()

,id,keyword,location,text,target
1971,2836,cyclone,NaN,1970 Mercury Cyclone GT Hood Moulding Very NIC...,0
2502,3593,desolate,NaN,The Desolate Hope: Part 2: MIRAD: http://t.co/...,0
5371,7663,panic,NaN,we was all just laughing and talking junk now ...,0
6258,8942,storm,NaN,FINALLY a storm,0
549,798,battle,NaN,Dragon Ball Z: Battle Of Gods (2014) - Rotten ...,0


In [4]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(df_train_shuffled['text'].to_numpy(),
                                                                           df_train_shuffled['target'].to_numpy(),
                                                                           train_size=0.9,
                                                                           random_state=123)

In [5]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [6]:
for i in range(0,10):
    print(f"Train sentences:{train_sentences[i]}, {train_labels[i]}")


Train sentences:@ACOUSTICMALOLEY no he was blazing it, 0
Train sentences:She looked back &amp; her daughter &amp; said 'everyone loved the picture I posted of you' &amp; like collided into another car like what the, 0
Train sentences:Couples having less sex... for fear it'll be a let down: Internet movies and books saying how sex 'ought to be' pÛ_ http://t.co/c1xhIzPrAd, 0
Train sentences:I just checked in! ÛÒ at On Fire on @ZomatoAUS #LoveFood http://t.co/9l5kqykrbG, 1
Train sentences:This setlist from @collideworship_ this past Sunday was powerful! What song was your favorite? http://t.co/vNzyBFGZcm, 0
Train sentences:Strict liability in the context of an airplane accident: Pilot error is a common component of most aviation cr... http://t.co/6CZ3bOhRd4, 1
Train sentences:Wow what beauty lies within every storm. Taken today after a rainstorm at A&amp;B Pipeliners http://t.co/pSt5bBQ0av, 1
Train sentences:beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/Ghmm

In [7]:
for i in range(0,10):
    print(f"Train sentences:{train_sentences[i]}, {train_labels[i]}")


Train sentences:@ACOUSTICMALOLEY no he was blazing it, 0
Train sentences:She looked back &amp; her daughter &amp; said 'everyone loved the picture I posted of you' &amp; like collided into another car like what the, 0
Train sentences:Couples having less sex... for fear it'll be a let down: Internet movies and books saying how sex 'ought to be' pÛ_ http://t.co/c1xhIzPrAd, 0
Train sentences:I just checked in! ÛÒ at On Fire on @ZomatoAUS #LoveFood http://t.co/9l5kqykrbG, 1
Train sentences:This setlist from @collideworship_ this past Sunday was powerful! What song was your favorite? http://t.co/vNzyBFGZcm, 0
Train sentences:Strict liability in the context of an airplane accident: Pilot error is a common component of most aviation cr... http://t.co/6CZ3bOhRd4, 1
Train sentences:Wow what beauty lies within every storm. Taken today after a rainstorm at A&amp;B Pipeliners http://t.co/pSt5bBQ0av, 1
Train sentences:beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/Ghmm

In [8]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 20 # max length our sequence will be (how many words from a Tweet does a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode='int',
                                   output_sequence_length=max_length)

In [9]:
# # Fit the text vectorizer
text_vectorizer.adapt(train_sentences)

In [10]:
sample_sentence = "There's a flood in my street and I don't why sometimes I felt like I'm the worst stupid"
text_vectorizer(sample_sentence)

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([ 280,    3,  227,    4,   13,  734,    7,    8,   58,   90, 1259,
          8, 1216,   25,   32,    2,  788, 1750,    0,    0])>

In [11]:
import random
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized Version:")
text_vectorizer(random_sentence)

Original text:
 A Warcraft 3-inspired mode is likely coming to Heroes of the Storm http://t.co/wz55NBYAO3 http://t.co/L7pMDmeJs1 http://t.co/mDP2nI1pQU        

Vectorized Version:


<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([   3, 3241, 6306,  996,    9, 1001,  257,    5, 2087,    6,    2,
         86,    1,    1,    1,    0,    0,    0,    0,    0])>

In [12]:
from tensorflow.keras import layers

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                                     output_dim=128,
                                     input_length=max_length) # how long is each input

embedding

In [13]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Origin text:\n {random_sentence}\
    \n\nVectorized Version:")
print(text_vectorizer(random_sentence))
print(f"\n\nEmbedded version:")
# Embedded the random (turn it into dense vectors)
sample_embed = embedding(text_vectorizer(random_sentence))
print(sample_embed)

Origin text:
 Add Familia to the arson squad.    

Vectorized Version:
tf.Tensor(
[2194 5633    5    2  634 1462    0    0    0    0    0    0    0    0
    0    0    0    0    0    0], shape=(20,), dtype=int64)


Embedded version:
tf.Tensor(
[[ 0.03811968 -0.01166002 -0.03636484 ...  0.03903282 -0.00371268
  -0.01210587]
 [ 0.0213904   0.04143325  0.02705984 ... -0.02512171 -0.03971826
   0.00097919]
 [-0.01713566 -0.04623553  0.04786595 ...  0.00504364 -0.00189174
   0.02054005]
 ...
 [-0.00352405 -0.0302776   0.04658551 ... -0.00467976  0.03969062
  -0.01387738]
 [-0.00352405 -0.0302776   0.04658551 ... -0.00467976  0.03969062
  -0.01387738]
 [-0.00352405 -0.0302776   0.04658551 ... -0.00467976  0.03969062
  -0.01387738]], shape=(20, 128), dtype=float32)


In [14]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
import sklearn

def calculate_results(y_true,y_pred):
    #Calcuate model accuracy
    model_accuracy = sklearn.metrics.accuracy_score(y_true,y_pred)*100
    # Calculate model precision,recall and
    model_precision, model_recall, model_f1, _ = sklearn.metrics.precision_recall_fscore_support(y_true,y_pred,average='weighted')
    model_results = {"accuracy": model_accuracy,
                    "precision": model_precision*100,
                     "recall": model_recall*100,
                    "f1_score": model_f1*100}
    return model_results


In [15]:
from helper_functions import create_tensorboard_callback
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimentional strings
x = text_vectorizer(inputs)
x = embedding(x) # create an embedding of the numberrized inputs
x = layers.Dense(1,activation='sigmoid')(x)
outputs = layers.GlobalAveragePooling1D()(x)
model_1 = tf.keras.Model(inputs,outputs, name='model_1_dense')

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 20)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 20, 128)           1280000   
                                                                 
 dense (Dense)               (None, 20, 1)             129       
                                                                 
 global_average_pooling1d (G  (None, 1)                0         
 lobalAveragePooling1D)                                          
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [16]:
# Compile model
model_1.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

In [17]:
history_1 = model_1.fit(x=train_sentences,
                       y=train_labels,
                       epochs=5,
                       validation_data=(val_sentences,val_labels),
                       callbacks=[create_tensorboard_callback(dir_name='tensorboard',
                                                             experiment_name='model_1_dense')])

Saving TensorBoard log files to: tensorboard/model_1_dense/20220616-183250
Epoch 1/5
215/215 [==============================] - 5s 6ms/step - loss: 0.6331 - accuracy: 0.6469 - val_loss: 0.5727 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5362 - accuracy: 0.7819 - val_loss: 0.5386 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4970 - accuracy: 0.8132 - val_loss: 0.5333 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4827 - accuracy: 0.8159 - val_loss: 0.5372 - val_accuracy: 0.7979
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4768 - accuracy: 0.8256 - val_loss: 0.5353 - val_accuracy: 0.7874
